# Kubeflow Pipelines

To create a Kubeflow pipeline, you need to define the pipeline components, which are the building blocks of your workflow. Each component is a containerized application that performs a specific task, such as data preprocessing, model training, or evaluation. You can use the Kubeflow Pipelines SDK to define and deploy your pipeline components.

You will also need to connect your KubeFlow Pipeline client to the Kubeflow Pipelines API server. This can be done by creating a KfpClient instance and providing the necessary authentication information, such as a bearer token or a Kubernetes service account token.

In [ ]:
import os
os.environ["KF_PIPELINES_SA_TOKEN_PATH"] = "/var/run/secrets/kubeflow/pipelines/token"

In [ ]:
import kfp

kfp_client = kfp.Client(host="http://ml-pipeline.kubeflow.svc.cluster.local:8888", existing_token=token)

In [ ]:
namespace = "aida-workshop"

In [ ]:
print(kfp_client.list_experiments(namespace=namespace))

## Download Spleen Decathlon Dataset

This cell executes the KubeFlow Pipeline to download the Spleen Decathlon dataset in the `/home/maia-user/shared` folder.

Other Decathlon tasks available are:
- Task01_BrainTumor
- Task02_Heart
- Task03_Liver
- Task04_Hippocampus
- Task05_Prostate
- Task06_Lung
- Task07_Pancreas
- Task08_HepaticVessel
- Task09_Spleen
- Task10_Colon

Check at [Decathlon Challenge Dataset](http://medicaldecathlon.com/) for more details about the single tasks.

In [ ]:
kfp_client.create_run_from_pipeline_package('KubeFlow/Pipelines/Decathlon_Dataset_pipeline.yaml', namespace="aida-workshop",
                                            arguments={"decathlon_task":"Task09_Spleen"}
                                           )

## Convert NIFTI to DICOM

In [ ]:
name_id = "spleen_1"
root_folder = "/home/maia-user/shared/Task09_Spleen"

modality = "CT"
study_description = "Spleen CT Study"
series_number = "301"

In [ ]:
import SimpleITK as sitk
import os
from pydicom.uid import generate_uid



# --- Input/Output ---
input_nifti = f"{root_folder}/imagesTs/{name_id}.nii.gz"
output_dir = f"{root_folder}/DICOM/{name_id}"
os.makedirs(output_dir, exist_ok=True)

# --- Read NIfTI ---
image = sitk.ReadImage(input_nifti)
# --- Spacing, direction, origin ---
spacing = image.GetSpacing()       # (x, y, z)
direction = image.GetDirection()   # 3x3 matrix as flat tuple
origin = image.GetOrigin()         # (x0, y0, z0)

# Convert direction cosines into orientation string
# DICOM wants a 6-value vector: first row + second row of the rotation matrix
orientation = [
    direction[0], direction[3], direction[6],  # row 1
    direction[1], direction[4], direction[7]   # row 2
]

# --- Writer ---
writer = sitk.ImageFileWriter()
writer.KeepOriginalImageUIDOn()

# --- Study/Series UIDs ---
study_uid = generate_uid()
series_uid = generate_uid()

# --- Custom metadata ---
series_tag_values = [
    ("0008|0060", modality),            # Modality
    ("0010|0010", name_id),      # PatientName
    ("0010|0020", name_id),        # PatientID
    ("0020|000D", study_uid),       # StudyInstanceUID
    ("0020|000E", series_uid),      # SeriesInstanceUID
    ("0008|1030", study_description),        # StudyDescription
    ("0008|103E", modality),  # SeriesDescription
]

# --- Write slice by slice ---
size = image.GetSize()
for i in range(size[2]):  # Loop over slices in Z
    slice_i = image[:, :, i]
    slice_i = sitk.Cast(slice_i, sitk.sitkInt16)
    # Compute ImagePositionPatient (origin + offset)
    position = [
        origin[0] + i * spacing[2] * direction[6],
        origin[1] + i * spacing[2] * direction[7],
        origin[2] + i * spacing[2] * direction[8],
    ]

    # Attach metadata
    for tag, value in series_tag_values:
        slice_i.SetMetaData(tag, value)

    # Geometry
    slice_i.SetMetaData("0028|0030", f"{spacing[0]}\\{spacing[1]}")  # PixelSpacing
    slice_i.SetMetaData("0020|0013", f"{i+1}")  # InstanceNumber
    slice_i.SetMetaData("0020|0011", series_number)  # SeriesNumber
    slice_i.SetMetaData("0018|0050", str(spacing[2]))                # SliceThickness
    slice_i.SetMetaData("0020|0032", "\\".join(map(str, position)))  # ImagePositionPatient
    slice_i.SetMetaData("0020|0037", "\\".join(map(str, orientation)))  # ImageOrientationPatient

    # Unique SOP Instance UID per slice

    slice_i.SetMetaData("0008|0018", str(generate_uid()))
    # Save file
    writer.SetFileName(os.path.join(output_dir, f"slice_{i:03d}.dcm"))
    writer.Execute(slice_i)

print(f"DICOM series written to {output_dir}")

## Upload DICOM Dataset to MinIO

In [ ]:
%%bash

export PATH=$PATH:$HOME/minio-binaries/
mc mb minio/spleen

In [ ]:
%%bash
cd /home/maia-user/shared/Task09_Spleen/ && zip -r Spleen_DICOM.zip DICOM/

In [ ]:
%%bash

export PATH=$PATH:$HOME/minio-binaries/
mc cp --recursive /home/maia-user/shared/Task09_Spleen/Spleen_DICOM.zip minio/spleen/